In [1]:
# Multiple outputs per cell in Jupyter 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%%javascript
// Evitar autoscroll.
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [25]:
# a = [1,2,3]
# b = [4,5,6]
# zipped_list = zip(a,b)
zipped_list=[(1, 4), (3, 5), (2, 6)]
zipped_list
unzipped_list=list(zip(*zipped_list))
unzipped_list
list(unzipped_list[0])
list(unzipped_list[1])

[(1, 4), (3, 5), (2, 6)]

[(1, 3, 2), (4, 5, 6)]

[1, 3, 2]

[4, 5, 6]

In [36]:
p=[(1, 4), (3, 5), (2, 6)]

p.sort()
p

p=list(zip(*p))
p

p=[list(p[0]),list(p[1])]
p

[(1, 4), (2, 6), (3, 5)]

[(1, 2, 3), (4, 6, 5)]

[[1, 2, 3], [4, 6, 5]]

In [249]:
import numpy as np
# from toolbox import *
import toolbox as tb
from skeleton import *
import pandas as pd
from gensim.models import FastText
%load_ext autoreload
%autoreload 2
import os
import tqdm
from microtc.utils import tweet_iterator
import datetime
from collections import Counter
import gc
from collections import defaultdict

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [227]:
class Tokens():
    tokenized_pointer=None
    
    def tokenize(self,json_file,npy_file=None,replace=False):
        self.npy_file=str(os.path.splitext(json_file)[0]+".npy") if not npy_file else npy_file

        if not replace and os.path.isfile(self.npy_file):
            print(f"** Replace is off. {os.path.abspath(self.npy_file)} already exists, then load.")
        else:
            tokenized_docs=[]
            self.N=0
            print(f"** Processing {json_file} ...")
            tx = datetime.datetime.now()
            for idx, tw in enumerate(tweet_iterator(json_file)):
                tb.show_progress(1000, tx, idx)
                twTokens = tb.process_line(tw['text']) # Tokenizando tweet.
                tokenized_docs.append(twTokens)
            self.N=idx+1
        
            maxLen=len(max(tokenized_docs))
            for i, doc in enumerate(tokenized_docs):
                tokenized_docs[i]=[doc[x] if x<len(doc) else '' for x in range(maxLen)]
            
            np.save(self.npy_file,tokenized_docs)
            del(tokenized_docs)
            gc.collect()
            print(f"** Processed {self.N} lines. Saved to {os.path.abspath(self.npy_file)}.")
        
        self.pointer=np.load(self.npy_file, mmap_mode='r')
        return self
    
    def get(self,n):
        return [x for x in self.pointer[n,:] if x != '']

In [229]:
tokens=Tokens().tokenize("data/geo-mx-2004_min.json",replace=True)

** Processing data/geo-mx-2004_min.json ...
2020-05-12 03:23:16.540113 :: 0.000166 - Processing item #0 
** Processed 10 lines. Saved to /home/jovyan/public/B6_ProyectoIntegrador/data/geo-mx-2004_min.npy.


In [237]:
for i in range(tokens.N):
    print(tokens.get(i))

['nochecit']
['teo', 'pai', 'pai', 'canon', 'nunc', 'crei', 'viv', 'moment', 'exist', 'graci']
['jajajaj', 'sab', 'sup', 'chid', 'manej', 'mism', 'user', 'igual', 'amig']
['obvi', 'hoy', 'cumpl', 'anos', 'apag', 'voz']
['jajajajaj', 'campanaz']
['hoy', 'cumpl', 'ano', 'plan', 'des', 'secret', 'gent', 'sabi', 'haci', 'sent']
['dij', 'amig', 'analiz', 'porn', 'tan', 'satur', 'wey', 'buenis', 'gent', 'normal']
['dan', 'ris', 'mem', 'vat', 'tres', 'pel', 'barb', 'si', 'jajajaj', 'sal']
['si', 'domicili', 'solicit', 'oblig', 'solidari', 'mism', 'proced', 'mand', 'dos', 'comprob']
['amig', 'seguidor', 'tod', 'person', 'respet', 'jueg', 'decision', 'jug', 'pued', 'influ']


In [256]:
class Index():
#     vocab = None
    N = 0
    DF = Counter({})
    TF = []
    
#     def __init__(self):
#         return self

    def computePostingLists(self, tokens, showProgressEach=None):
        print("** Processing ...")
        tx = datetime.datetime.now()
        for idx in range(tokens.N):
            tb.show_progress(showProgressEach,tx,idx)
            
            tok=tokens.get(idx)
            # Sumando frecuencias individiales.
            twCnt = Counter(tok)
            
            self.TF.append(twCnt)
            
            # Sumando frecuencias por documento (DF).
            self.DF.update(list(twCnt.keys()))

            # Contabilizando documentos.
            self.N += 1
            
#         words = self.DF.keys()
#         self.vocab = dict(zip(words, range(len(words))))
#         self.DF_vec = cnt2vec(self.DF, self.vocab)
#         self.IDF_vec = np.log2(self.N / (self.DF_vec + 1))
        
#         # Calculando TF-IDF y asignando a usuarios.
#         for userId in self.users:
#             TF_cnt = self.users[userId]
#             self.users[userId] = cnt2vec(TF_cnt, self.vocab) * self.IDF_vec
        postlists=defaultdict(list)
    
        for i,c in enumerate(self.TF):
            for w,tf in c.items():
                postlists[w].append((tf*np.log2(self.N/(self.DF[w]+1)),i))
        
        print(postlists)
        for p in postlists:
            p.sort()
            p=list(zip(*p))
            p=[list(p[1]),list(p[0])]
        return self

    def getX(self):
        return np.array([self.users[key] for key in self.users])
    
    def cluster(self, n_clusters=9):
        self.cluster = KMeans(n_clusters=n_clusters, n_jobs=4).fit(self.getX())
        return self.cluster

In [257]:
index_=Index().computePostingLists(tokens,showProgressEach=10)
index_


** Processing ...
2020-05-12 03:44:07.180412 :: 1.2e-05 - Processing item #0 


AxisError: axis -1 is out of bounds for array of dimension 0

In [238]:
cnt=Counter(['a','b','a','c','c','c'])
cnt

Counter({'a': 2, 'b': 1, 'c': 3})

In [255]:
cnt['a']

2

In [234]:
import sys
local_vars = list(locals().items())
tot=0
vars_=[]
for var, obj in local_vars:
    mem=sys.getsizeof(obj)
    tot+=mem
    vars_.append((mem,var))
print("Total =",tot)
vars_.sort(reverse=True)
print(vars_)

Total = 132888
[(3512, 'local_vars'), (3312, 'vars_'), (2904, 'stop_words'), (2288, '_oh'), (2288, 'Out'), (2224, '_ih'), (2224, 'In'), (1867, '_i227'), (1862, '_i224'), (1856, '_i210'), (1848, '_i208'), (1846, '_i212'), (1757, '_i206'), (1656, '_i204'), (1628, '_i202'), (1623, '_i200'), (1573, '_i198'), (1516, '_i53'), (1438, '_i51'), (1436, '_i195'), (1433, '_i49'), (1431, '_i192'), (1431, '_i190'), (1429, '_i188'), (1429, '_i187'), (1423, '_i47'), (1403, '_i181'), (1402, '_i184'), (1402, '_i183'), (1382, '_i179'), (1345, '_i176'), (1345, '_i173'), (1335, '_i45'), (1332, '_i43'), (1284, '_i41'), (1281, '_i59'), (1247, '_i57'), (1245, '_i56'), (1236, '_i39'), (1229, '_i37'), (1229, '_i36'), (1192, 'punctuation_table'), (1183, '_i34'), (1166, '_i26'), (1159, '_i31'), (1159, '_i27'), (1076, '_i24'), (1076, '_i22'), (1076, '_i19'), (1076, '_i16'), (1076, '_i13'), (1064, 'skmeans'), (1064, 'kNN'), (1064, 'Tokens'), (1064, 'PCA'), (1064, 'NearestCentroid'), (1064, 'InteractiveShell'), (106